# Packages

In [18]:
using Pkg; Pkg.activate("."); 
using CSV
using MUST  # For the models
using TSO   # For the EoS
using DelimitedFiles
using DataFrames
using Glob;

  Activating project at `/mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/examples/initial_models`


In [19]:
eos = reload(SqEoS, "/home/eitner/shared/TS_opacity_tables/opacity_tables/DIS_MARCS_v0.5.5.1/eos.hdf5");

# Infrastructure

In [20]:
struct StaggerGrid{DF<:DataFrame}
    name :: String
    info :: DF
end

we first create an info csv file, that will represent the Stagger grid

In [21]:
folder = "/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/"
listOfFolders = [f[1:end-1] for f in glob("*/", folder) if isdir(f)]
listOfFolderNames = [f[first(findlast("/", f))+1:end] for f in listOfFolders]

FolderNamesMask   = [f[1]=='t' for f in listOfFolderNames]
listOfFolders     = listOfFolders[FolderNamesMask]
listOfFolderNames = listOfFolderNames[FolderNamesMask];

In [22]:
## only for Remo sub-folder

#for (i, f) in enumerate(listOfFolders)
#    f_intern = glob("*/", f)
#    f_intern = f_intern[isdir.(f_intern)]
#    if length(f_intern) > 1
#        @warn "there are more than one entried in $(f)"
#    end
#    f_intern = f_intern |> first
#   
#    listOfFolders[i] = f_intern
#end

In [23]:
meshFiles = ["" for _ in listOfFolders]
snapshots = ["" for _ in listOfFolders]
isPresent  = trues(length(meshFiles))

for (i, f) in enumerate(listOfFolders)
    mesh_file = glob("*.msh", f)
    
    mesh_file = if length(mesh_file) == 0
        #@warn "No mesh file in $(f)"
        isPresent[i] = false
        nothing
    elseif length(mesh_file) > 1
        @warn "More than one mesh file present in $(f)"
        fist(mesh_file)
    else
        first(mesh_file)
    end

    meshFiles[i] = isnothing(mesh_file) ?  "" : basename(mesh_file)



    snap_file = glob("*.aux", f)
    
    snap_file = if length(snap_file) == 0
        #@warn "No snap file in $(f)"
        isPresent[i] = false
        nothing
    elseif length(snap_file) > 1
        #@warn "More than one snap file present in $(f)"
        rand(snap_file)
    else
        fist(snap_file)
    end

    snapshots[i] = isnothing(snap_file) ? "" : basename(snap_file)[1:findlast('.', snap_file |> basename)-1]
end


info = DataFrame(Dict("name"=>listOfFolderNames[isPresent], "folder"=>listOfFolders[isPresent], "mesh"=>meshFiles[isPresent], "snapshot"=>snapshots[isPresent]));

In [24]:
info

Row,folder,mesh,name,snapshot
,String,String,String,String
1,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t45g20m00,t45g20m00.msh,t45g20m00,t45g20m00_00168
2,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t45g20m20,t45g20m20.msh,t45g20m20,t45g20m20_00031
3,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t45g20m40,t45g20m40.msh,t45g20m40,t45g20m40_00010
4,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t45g40m00,t45g40m00.msh,t45g40m00,t45g40m00_00042
5,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t55g45m00,t55g45m00.msh,t55g45m00,t55g45m00_00000
6,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t5777g44m00,t5777g44m0005.msh,t5777g44m00,t5777g44m0005_00070
7,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t60g45m00,t60g45m00.msh,t60g45m00,t60g45m00_00068
8,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t65g40m20,t65g40m20.msh,t65g40m20,t65g40m20_00052
9,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t65g40m40,t65g40m40.msh,t65g40m40,t65g40m40_00027


Now we add information of the models and the preferred snapshot etc.

In [25]:
function model_properties(name, path, snap_name)
    ## we get the temperature and logg from the name only at this stage
    teff =  parse(Float64, name[2:findfirst('g', name)-1]) .*100
    logg =  parse(Float64, name[findfirst('g', name)+1:findfirst('m', name)-1]) ./10
    feh  = -parse(Float64, name[findfirst('m', name)+1:end]) ./10

    b = MUST.Box(MUST.StaggerSnap(snap_name, path))
    #MUST.save(b, name="box_stagger_$(snap_name)", folder="")

    T = MUST.plane_statistic(MUST.mean, b, :T)
    d = MUST.plane_statistic(MUST.mean, b, :d)
    z = MUST.axis(b, :z)

    resolution(model) = minimum(abs.(diff(MUST.axis(model, :z))))
    resolution_horizontal(model) = minimum(abs.(diff(MUST.axis(model, :x))))
	Δt(R, u, c=1.0) = c * R / u
    scaling(δt, goal_scale=1e-3) = exp10.(round(log10(δt / goal_scale), sigdigits=1))

    vmax = max(abs(maximum(b[:ux])), 
				abs(maximum(b[:uy])), 
				abs(maximum(b[:uz])))

    s = scaling(Δt(resolution(b), vmax, 0.1))

    #initial_model = SimpleInitialModel(teff, logg, z, T, log.(d))
    
    open("$(snap_name)_av.dat", "w") do f
        writedlm(f, [z T log.(d)])
    end

    ## save return the additional column values
    return (teff, logg, feh, 
            minimum(b[:x]), maximum(b[:x]), 
            minimum(b[:y]), maximum(b[:y]), 
            minimum(b[:z]), maximum(b[:z]), 
            min(minimum(b[:ux]), minimum(b[:uy]), minimum(b[:uz])),
            max(maximum(b[:ux]), maximum(b[:uy]), maximum(b[:uz])),
            s,
            resolution_horizontal(b),
            "$(snap_name)_av.dat")
end

model_properties (generic function with 1 method)

In [26]:
teff, logg, feh, mi_x, ma_x, mi_y, ma_y, mi_z, ma_z, vmin, vmax, tscale, hres, av_path = [], [], [], [], [], [], [], [], [], [], [], [], [], []

for i in 1:nrow(info)
    @info "processing $(info[i, "name"])"
    res = model_properties(info[i, "name"], info[i, "folder"], info[i, "snapshot"])

    append!(teff,    [res[1]])
    append!(logg,    [res[2]])
    append!(feh,     [res[3]])
    append!(mi_x,    [res[4]])
    append!(ma_x,    [res[5]])
    append!(mi_y,    [res[6]])
    append!(ma_y,    [res[7]])
    append!(mi_z,    [res[8]])
    append!(ma_z,    [res[9]])
    append!(vmin,    [res[10]])
    append!(vmax,    [res[11]])
    append!(tscale,  [res[12]])
    append!(hres,    [res[13]])
    append!(av_path, [res[14]])
end

┌ Info: processing t45g20m00
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 440
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


┌ Info: processing t45g20m20
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 351
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


┌ Info: processing t45g20m40
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 440
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


┌ Info: processing t45g40m00
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 445
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


┌ Info: processing t55g45m00
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 411
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


...

┌ Info: processing t60g45m00
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 453
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


┌ Info: processing t65g40m20
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 636
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


┌ Info: processing t65g40m40
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 579
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


┌ Info: processing t65g45m00
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 465
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


┌ Info: processing t70g45m00
└ @ Main /home/eitner/shared/model_grid/examples/initial_models/from_stagger.ipynb:4
┌ Info: Increase number of points from 238 to 510
└ @ MUST /mnt/beegfs/gemini/groups/bergemann/users/eitner/model_grid/MUST.jl/src/_atmos2legacy.jl:173


In [27]:
info[!, "teff"]    = teff
info[!, "logg"]    = logg
info[!, "feh"]     = feh
info[!, "mi_x"]    = mi_x
info[!, "ma_x"]    = ma_x
info[!, "mi_y"]    = mi_y
info[!, "ma_y"]    = ma_y
info[!, "mi_z"]    = mi_z
info[!, "ma_z"]    = ma_z
info[!, "vmin"]    = vmin
info[!, "vmax"]    = vmax
info[!, "tscale"]  = tscale
info[!, "hres"]    = hres
info[!, "av_path"] = av_path;

In [28]:
CSV.write("stagger_grid.mgrid", info)

"stagger_grid.mgrid"

In [29]:
# copy the average profiles to the source folders
for i in 1:nrow(info)
    av_name = basename(info[i, "av_path"])
    dest    = joinpath(info[i, "folder"], av_name)

    #cp(info[i, "av_path"], dest)
end

# Load the Grid

In [30]:
StaggerGrid(path) = begin
    info = CSV.read(path, DataFrame)
    name = basename(path)[1:findlast('.', basename(path))-1]

    StaggerGrid(name, info)
end

StaggerGrid

In [31]:
grid = StaggerGrid("stagger_grid.mgrid");

In [32]:
grid.info

Row,folder,mesh,name,snapshot,teff,logg,feh,mi_x,ma_x,mi_y,ma_y,mi_z,ma_z,vmin,vmax,tscale,hres,av_path
,String,String31,String15,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String31
1,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t45g20m00,t45g20m00.msh,t45g20m00,t45g20m00_00168,4500.0,2.0,-0.0,0.0,2.39e11,0.0,2.39e11,-8.85898e10,2.62912e10,-2.38794e6,2.85569e6,10000.0,9.99997e8,t45g20m00_00168_av.dat
2,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t45g20m20,t45g20m20.msh,t45g20m20,t45g20m20_00031,4500.0,2.0,-2.0,0.0,1.69292e11,0.0,1.69292e11,-4.62167e10,1.93811e10,-1.33372e6,1.84548e6,10000.0,7.08313e8,t45g20m20_00031_av.dat
3,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t45g20m40,t45g20m40.msh,t45g20m40,t45g20m40_00010,4500.0,2.0,-4.0,0.0,1.49375e11,0.0,1.49375e11,-5.08822e10,2.16435e10,-1.36513e6,2.62862e6,10000.0,6.24984e8,t45g20m40_00010_av.dat
4,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t45g40m00,t45g40m00.msh,t45g40m00,t45g40m00_00042,4500.0,4.0,-0.0,0.0,1.39417e9,0.0,1.39417e9,-5.99664e8,1.69718e8,-8.94969e5,8.70332e5,100.0,5.83322e6,t45g40m00_00042_av.dat
5,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t55g45m00,t55g45m00.msh,t55g45m00,t55g45m00_00000,5500.0,4.5,-0.0,0.0,6.97083e8,0.0,6.97083e8,-2.28791e8,8.38153e7,-1.13401e6,1.14704e6,100.0,2.91661e6,t55g45m00_00000_av.dat
6,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t5777g44m00,t5777g44m0005.msh,t5777g44m00,t5777g44m0005_00070,577700.0,4.4,-0.0,0.0,7.96667e8,0.0,7.96667e8,-2.93971e8,9.57648e7,-1.32773e6,1.58787e6,100.0,3.33328e6,t5777g44m0005_00070_av.dat
7,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t60g45m00,t60g45m00.msh,t60g45m00,t60g45m00_00068,6000.0,4.5,-0.0,0.0,6.97083e8,0.0,6.97083e8,-2.63195e8,8.59617e7,-1.40654e6,1.48123e6,100.0,2.91661e6,t60g45m00_00068_av.dat
8,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t65g40m20,t65g40m20.msh,t65g40m20,t65g40m20_00052,6500.0,4.0,-2.0,0.0,1.99167e9,0.0,1.99167e9,-1.0953e9,3.0781e8,-1.83404e6,2.65585e6,100.0,8.33318e6,t65g40m20_00052_av.dat
9,/mnt/beegfs/gemini/groups/bergemann/users/shared-storage/bergemann-data/Stagger_zaz/grid/t65g40m40,t65g40m40.msh,t65g40m40,t65g40m40_00027,6500.0,4.0,-4.0,0.0,2.19083e9,0.0,2.19083e9,-1.07635e9,3.31133e8,-2.22901e6,3.88219e6,100.0,9.16646e6,t65g40m40_00027_av.dat


# Look at the average profiles

In [33]:
Models(grid::StaggerGrid) = begin 
    [Average3D(grid.info[i, "av_path"], logg=grid.info[i, "logg"]) for i in 1:nrow(grid.info)]
end

Models (generic function with 1 method)

In [34]:
models = Models(grid);